In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import cvxpy as cp
from UsefulFunctions import Optimizer_NonProsumer

## Task 2

### 2.1

In [ ]:
### Load electricity prices ###
price_path = os.path.join(os.getcwd(),'ElspotpricesEA.csv')
df_prices = pd.read_csv(price_path)

### Convert to datetime ###
df_prices["HourDK"] = pd.to_datetime(df_prices["HourDK"])
df_prices["HourUTC"] = pd.to_datetime(df_prices["HourUTC"])
df_prices['HourUTC'] = df_prices['HourUTC'].dt.tz_localize('UTC')
df_prices['HourDK'] = df_prices['HourUTC'].dt.tz_convert('CET')

### Convert prices to DKK/kWh ###
df_prices['SpotPriceDKK'] = df_prices['SpotPriceDKK']/1000

### Filter only DK2 prices, restricting price list to DK2 ###
df_prices = df_prices.loc[df_prices['PriceArea']=="DK2"]

### Keep only the local time and price columns ###
df_prices = df_prices[['HourDK','SpotPriceDKK']]

### Reset the index ###
df_prices = df_prices.reset_index(drop=True)

### Adding columns for year, month, day, hour as datetime objects ###

df_prices["Year"]=df_prices["HourDK"].dt.year
df_prices["Month"]=df_prices["HourDK"].dt.month
df_prices["Day"]=df_prices["HourDK"].dt.day_of_year
df_prices["Hour"]=df_prices["HourDK"].dt.hour

### Filtering by year ###

df_prices_2019 = df_prices.loc[df_prices["Year"].isin([2019])]
df_prices_2020 = df_prices.loc[df_prices["Year"].isin([2020])]
df_prices_2021 = df_prices.loc[df_prices["Year"].isin([2021])]
df_prices_2022 = df_prices.loc[df_prices["Year"].isin([2022])]
df_prices_2023 = df_prices.loc[df_prices["Year"].isin([2023])]

### Establishing the parameters of the battery ###

# Battery parameters
battery_params = {
    'Pmax': 5,      # Power capacity in kW
    'Cmax': 10,     # Energy capacity in kWh
    'Cmin': 1,
    'C_0': 5,       # Initial SOC (50%)
    'C_n': 5,       # Final SOC (50%)
    'n_c': 0.95,    # Charging efficiency
    'n_d': 0.95     # Discharging efficiency
}
###Dictionary for optimization results of each year###
costs_only = {'2019':0, '2020':0, '2021':0, '2022': 0, '2023': 0}

### Finding mean prices for each day of the year ###
years = [df_prices_2019, df_prices_2020, df_prices_2021, df_prices_2022, df_prices_2023]    

for year in years:
    year_value = year["Year"].values[0]
    for day in year["Day"].unique():
        prices_day = year[year["Day"] == day]
        price = prices_day["SpotPriceDKK"].values
        cost_year, pc_value, pd_value, X_value = Optimizer_NonProsumer(battery_params, price)
        costs_only[str(year_value)] += cost_year

      
##Notes for signs of cost in DKK/kWh:
#When the result of cost is positive we're making money, and when it is the opposite, we're losing it.     

# Display results
print('--- 2.1 results ---')
print(f"Annual costs with Storage [DKK/kWh]:")
print(f"2019: {costs_only['2019']:.2f}")
print(f"2020: {costs_only['2020']:.2f}")
print(f"2021: {costs_only['2021']:.2f}")
print(f"2022: {costs_only['2022']:.2f}")
print(f"2023: {costs_only['2023']:.2f}")
